In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# current working directory
PATH = r"C:\Users\doosti\Dropbox (Chapman)\Research\Research Projects\Fitness\Data"

In [14]:
# Loading data files
trend = pd.read_csv(os.path.join(PATH, 'channel_trends_to_stata_performance_v2.csv'))

In [4]:
trend.head()

,creator_id,date,views,engagements,uploads,followers,avg_challenge,publish_date,challenge_dummy,total_views,...,high_engagement,high_engagement_per_upload,high_num_videos,high_followers,high_followers_growth,high_influencer_score,high_tenure,cum_uploads,cum_followers,cum_challenges
0,YhItxj8Ap9,2020-09-15,8173,153,0,0.0,NaN,NaN,0,22747130,...,True,False,True,True,False,False,True,0,0.0,0
1,YhItxj8Ap9,2020-09-16,9924,392,2,0.0,0.0,2020-09-16,0,22747130,...,True,False,True,True,False,False,True,2,0.0,0
2,YhItxj8Ap9,2020-09-16,9924,392,2,0.0,0.0,2020-09-16,0,22747130,...,True,False,True,True,False,False,True,4,0.0,0
3,YhItxj8Ap9,2020-09-17,9976,369,1,0.0,0.0,2020-09-17,0,22747130,...,True,False,True,True,False,False,True,5,0.0,0
4,YhItxj8Ap9,2020-09-18,9279,223,0,0.0,NaN,NaN,0,22747130,...,True,False,True,True,False,False,True,5,0.0,0


In [32]:
# sort the data based on the date for each creator, then calculate the challenge duration by counting the consecutive days that challenge is active
# challenge duration should reset to 0 after the challenge ends
trend = trend.sort_values(by=['creator_id', 'date'])
trend['challenge_duration'] = trend.groupby('creator_id')['challenge_dummy'].cumsum()*trend['challenge_dummy']

# Mark rows where a challenge starts and ends
trend['prev_challenge'] = trend.groupby('creator_id')['challenge_dummy'].shift(1).fillna(0)
trend['next_challenge'] = trend.groupby('creator_id')['challenge_dummy'].shift(-1).fillna(0)

trend['start_challenge'] = ((trend['challenge_dummy'] == 1) & (trend['prev_challenge'] == 0))
trend['end_challenge'] = ((trend['challenge_dummy'] == 1) & (trend['next_challenge'] == 0))


In [34]:
trend['challenge_id'] = (trend['start_challenge']).cumsum()

In [45]:
# Keep only the rows where a challenge is active
challenge_periods = trend[trend['challenge_dummy'] == 1]

# Aggregate to find the start and end dates for each challenge
challenge_durations = challenge_periods.groupby(['creator_id', 'challenge_id']).agg(start_date=('date', 'min'), end_date=('date', 'max'))

# Calculate duration of each challenge
challenge_durations['duration'] = (pd.to_datetime(challenge_durations['end_date']) - pd.to_datetime(challenge_durations['start_date'])).dt.days + 1

In [37]:
challenge_durations.head()

start_date    end_date  duration
creator_id          challenge_id                                  
262h9yTG1f03Nobo501 1             2020-11-06  2020-11-25        20
                    2             2020-12-09  2020-12-09         1
                    3             2020-12-13  2020-12-13         1
                    4             2020-12-18  2020-12-18         1
                    5             2020-12-20  2020-12-20         1

In [38]:
trend = trend.merge(challenge_durations, on=['creator_id', 'challenge_id'], how='left')

In [39]:
trend.head()

,creator_id,date,views,engagements,uploads,followers,avg_challenge,publish_date,challenge_dummy,total_views,...,cum_challenges,challenge_duration,prev_challenge,next_challenge,start_challenge,end_challenge,challenge_id,start_date,end_date,duration
0,262h9yTG1f03Nobo501,2020-09-15,325,19,0,23.0,NaN,NaN,0,22309517,...,0,0,0.0,0.0,False,False,0,NaN,NaN,NaN
1,262h9yTG1f03Nobo501,2020-09-16,288,14,0,24.0,NaN,NaN,0,22309517,...,0,0,0.0,0.0,False,False,0,NaN,NaN,NaN
2,262h9yTG1f03Nobo501,2020-09-17,268,19,0,23.0,NaN,NaN,0,22309517,...,0,0,0.0,0.0,False,False,0,NaN,NaN,NaN
3,262h9yTG1f03Nobo501,2020-09-18,255,18,0,24.0,NaN,NaN,0,22309517,...,0,0,0.0,0.0,False,False,0,NaN,NaN,NaN
4,262h9yTG1f03Nobo501,2020-09-19,257,13,0,23.0,NaN,NaN,0,22309517,...,0,0,0.0,0.0,False,False,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84517,ybNsef2VSA,2023-09-10,165981,2763,1,0.0,0.0,2023-09-10,0,444123931,...,41,0,0.0,0.0,False,False,542,2023-01-08,2023-01-08,1.0
84518,ybNsef2VSA,2023-09-11,178664,3145,1,0.0,0.0,2023-09-11,0,444123931,...,41,0,0.0,0.0,False,False,542,2023-01-08,2023-01-08,1.0
84519,ybNsef2VSA,2023-09-12,180848,3038,1,0.0,0.0,2023-09-12,0,444123931,...,41,0,0.0,0.0,False,False,542,2023-01-08,2023-01-08,1.0
84520,ybNsef2VSA,2023-09-13,168870,3002,1,0.0,0.0,2023-09-13,0,444123931,...,41,0,0.0,0.0,False,False,542,2023-01-08,2023-01-08,1.0


In [83]:
# definig the post challenge period. post_challenge = 1 right after the challenge ends for the duration of the challenge
trend['post_challenge'] = 0
shifting_ind = trend[trend.challenge_dummy==1].index + trend[trend.challenge_dummy==1].duration
ind_creator = trend.groupby('creator_id').apply(lambda x: x.index.isin(shifting_ind))
for creator_id in ind_creator.index:
    global_ind = trend[trend.creator_id==creator_id].loc[ind_creator[creator_id]].index
    trend.loc[global_ind, 'post_challenge'] = 1

In [84]:
# save the data
trend.to_csv(os.path.join(PATH, 'channel_trends_to_stata_performance_v3.csv'), index=False)

In [49]:
df = pd.DataFrame({'creator_id':[1]*10 + [2]*10, 'date':pd.date_range(start='1/1/2018', periods=10).tolist()*2, 'challenge':[0,0,1,1,1,0,0,1,1,0]*2})

In [50]:
df.sort_values(by=['creator_id', 'date'], inplace=True)

df['prev_challenge'] = df.groupby('creator_id')['challenge'].shift(1).fillna(0)
df['next_challenge'] = df.groupby('creator_id')['challenge'].shift(-1).fillna(0)

df['start_challenge'] = ((df['challenge'] == 1) & (df['prev_challenge'] == 0))
df['end_challenge'] = ((df['challenge'] == 1) & (df['next_challenge'] == 0))

In [51]:
df['challenge_id'] = (df['start_challenge']).cumsum()

In [52]:
# Keep only the rows where a challenge is active
challenge_periods = df[df['challenge'] == 1]

# Aggregate to find the start and end dates for each challenge
challenge_durations = challenge_periods.groupby(['creator_id', 'challenge_id']).agg(start_date=('date', 'min'), end_date=('date', 'max'))

# Calculate duration of each challenge
challenge_durations['duration'] = (pd.to_datetime(challenge_durations['end_date']) - pd.to_datetime(challenge_durations['start_date'])).dt.days + 1

In [47]:
challenge_durations

start_date   end_date  duration
creator_id challenge_id                                
1          1            2018-01-03 2018-01-05         3
           2            2018-01-08 2018-01-09         2
2          3            2018-01-03 2018-01-05         3
           4            2018-01-08 2018-01-09         2

In [53]:
df = df.merge(challenge_durations.reset_index(), on=['creator_id', 'challenge_id'], how='left')

In [70]:
ind = df[df.challenge==1].index + df[df.challenge==1].duration

In [71]:
ind_creator = df.groupby('creator_id').apply(lambda x: x.index.isin(ind))

In [80]:
for creator_id in ind_creator.index:
    global_ind = df[df.creator_id==creator_id].loc[ind_creator[creator_id]].index
    df.loc[global_ind, 'post_challenge'] = 1

In [81]:
df

,creator_id,date,challenge,prev_challenge,next_challenge,start_challenge,end_challenge,challenge_id,start_date,end_date,duration,post_challenge
0,1,2018-01-01,0,0.0,0.0,False,False,0,NaT,NaT,NaN,0
1,1,2018-01-02,0,0.0,1.0,False,False,0,NaT,NaT,NaN,0
2,1,2018-01-03,1,0.0,1.0,True,False,1,2018-01-03,2018-01-05,3.0,0
3,1,2018-01-04,1,1.0,1.0,False,False,1,2018-01-03,2018-01-05,3.0,0
4,1,2018-01-05,1,1.0,0.0,False,True,1,2018-01-03,2018-01-05,3.0,0
5,1,2018-01-06,0,1.0,0.0,False,False,1,2018-01-03,2018-01-05,3.0,1
6,1,2018-01-07,0,0.0,1.0,False,False,1,2018-01-03,2018-01-05,3.0,1
7,1,2018-01-08,1,0.0,1.0,True,False,2,2018-01-08,2018-01-09,2.0,1
8,1,2018-01-09,1,1.0,0.0,False,True,2,2018-01-08,2018-01-09,2.0,0
9,1,2018-01-10,0,1.0,0.0,False,False,2,2018-01-08,2018-01-09,2.0,1
